<a href="https://colab.research.google.com/github/IlyasMoutawwakil/huggingface-course/blob/main/chapter-1-transformers-what-can-they-do.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transformers, what can they do?

Install the Transformers and Datasets libraries to run this notebook.

In [19]:
%%capture
!pip install datasets transformers[sentencepiece]

The most basic object in the 🤗 Transformers library is the pipeline() function. It connects a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer:

In [20]:
from transformers import pipeline

In [21]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

We can even pass several sentences!

In [22]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [23]:
classifier("That movie was pretty bad but i really liked it.")

[{'label': 'POSITIVE', 'score': 0.9988355040550232}]

By default, this pipeline selects a particular pretrained model that has been fine-tuned for sentiment analysis in English. The model is downloaded and cached when you create the classifier object. If you rerun the command, the cached model will be used instead and there is no need to download the model again.

There are three main steps involved when you pass some text to a pipeline:

The text is preprocessed into a format the model can understand.
The preprocessed inputs are passed to the model.
The predictions of the model are post-processed, so you can make sense of them.
Some of the currently available pipelines are:

- feature-extraction (get the vector representation of a text)
- fill-mask
- ner (named entity recognition)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification

Let’s have a look at a few of these!

### Zero-shot classification
We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled. <br>
This is a common scenario in real-world projects because annotating text is usually time-consuming and requires domain expertise. <br>
For this use case, the **zero-shot-classification** pipeline is very powerful: it allows you to specify which labels to use for the classification, so you don’t have to rely on the labels of the pretrained model. <br>
You’ve already seen how the model can classify a sentence as positive or negative using those two labels — but it can also classify the text using any other set of labels you like.

In [24]:
classifier2 = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [25]:
classifier2(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "movies", "sports"],
)

{'labels': ['movies', 'education', 'sports', 'politics'],
 'scores': [0.5095946192741394,
  0.4361685514450073,
  0.03181035816669464,
  0.022426392883062363],
 'sequence': 'This is a course about the Transformers library'}

This pipeline is called **zero-shot** because you don’t need to fine-tune the model on your data to use it. <br>
It can directly return probability scores for any list of labels you want!

### Text generation
Now let’s see how to use a pipeline to generate some text. <br>
The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text. <br>
This is similar to the predictive text feature that is found on many phones. <br> Text generation involves randomness, so it’s normal if you don’t get the same results as shown below.

In [26]:
generator = pipeline("text-generation")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


In [27]:
generator("In this course, we will teach you how to")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to design, analyze and design for web, mobile and web platforms. We will also see you in an interaction program.\n\nOur goal is to be a full time professional web developer who will be building'}]

You can control how many different sequences are generated with the argument **num_return_sequences** and the total length of the output text with the argument **max_length**.

### Using any model from the Hub in a pipeline
The previous examples used the default model for the task at hand, but you can also choose a particular model from the Hub to use in a pipeline for a specific task — say, text generation. <br>
Go to the [Model Hub](https://huggingface.co/models) and click on the corresponding tag on the left to display only the supported models for that task. <br>
You should get to a page like [this one](https://huggingface.co/models?pipeline_tag=text-generation).

Let’s try the [distilgpt2](https://huggingface.co/distilgpt2) model! Here’s how to load it in the same pipeline as before:

In [28]:
generator2 = pipeline("text-generation", model="distilgpt2")
generator2("In this course, we will teach you how to",
           max_length=30, num_return_sequences=2,)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to make light and light sense in your own work.\n\n\nThis course will teach you how to'},
 {'generated_text': 'In this course, we will teach you how to start a video game business.\n\n\n\nStep 1: Getting started\nWe will also test'}]

### Mask filling
The next pipeline you’ll try is **fill-mask**. <br>
The idea of this task is to fill in the blanks in a given text:

In [29]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models", top_k=2)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


[{'score': 0.1963154524564743,
  'sequence': 'This course will teach you all about mathematical models',
  'token': 30412,
  'token_str': ' mathematical'},
 {'score': 0.04449218139052391,
  'sequence': 'This course will teach you all about building models',
  'token': 745,
  'token_str': ' building'}]

The **top_k** argument controls how many possibilities you want to be displayed. <br> Note that here the model fills in the special **<"mask">** word, which is often referred to as a mask token. <br> 
Other mask-filling models might have different mask tokens, so it’s always good to verify the proper mask word when exploring other models. <br>
One way to check it is by looking at the mask word used in the widget.

### Named entity recognition
**Named entity recognition (NER)** is a task where the model has to find which parts of the input text correspond to entities such as persons, locations, or organizations. <br>
Let’s look at an example:

In [30]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:136: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


[{'end': 18,
  'entity_group': 'PER',
  'score': 0.9981694,
  'start': 11,
  'word': 'Sylvain'},
 {'end': 45,
  'entity_group': 'ORG',
  'score': 0.9796019,
  'start': 33,
  'word': 'Hugging Face'},
 {'end': 57,
  'entity_group': 'LOC',
  'score': 0.9932106,
  'start': 49,
  'word': 'Brooklyn'}]

Here the model correctly identified that Sylvain is a person (PER), Hugging Face an organization (ORG), and Brooklyn a location (LOC).

We pass the option **grouped_entities=True** in the pipeline creation function to tell the pipeline to regroup together the parts of the sentence that correspond to the same entity: here the model correctly grouped “Hugging” and “Face” as a single organization, even though the name consists of multiple words. <br>
In fact, as we will see in the next chapter, the preprocessing even splits some words into smaller parts. <br>
For instance, Sylvain is split into four pieces: S, ##yl, ##va, and ##in. 
In the post-processing step, the pipeline successfully regrouped those pieces.

### Question answering
The **question-answering** pipeline answers questions using information from a given context:

In [31]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [32]:
question_answerer(question="Where do I live?",
                  context="My name is Batuhan and I live in Turkey.")

{'answer': 'Turkey', 'end': 39, 'score': 0.980381429195404, 'start': 33}

Note that this pipeline works by extracting information from the provided context; it does not generate the answer.

### Summarization
**Summarization** is the task of reducing a text into a shorter text while keeping all (or most) of the important aspects referenced in the text. <br>
Here’s an example:

In [33]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [34]:
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

In [35]:
summarizer(
"""
After getting a green card in exchange for assassinating a Cuban government official, Tony Montana (Al Pacino) stakes a claim on the drug trade in Miami. 
Viciously murdering anyone who stands in his way, Tony eventually becomes the biggest drug lord in the state, controlling nearly all the cocaine that comes through Miami. 
But increased pressure from the police, wars with Colombian drug cartels and his own drug-fueled paranoia serve to fuel the flames of his eventual downfall.""")

Your max_length is set to 142, but you input_length is only 102. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': ' Tony Montana (Al Pacino) stakes a claim on the drug trade in Miami . He becomes the biggest drug lord in the state, controlling nearly all the cocaine that comes through Miami . But increased pressure from the police, wars with Colombian drug cartels and his own drug-fueled paranoia fuel the flames of his eventual downfall .'}]

Like with text generation, you can specify a **max_length** or a **min_length** for the result.

The pipelines shown so far are mostly for demonstrative purposes. <br>
They were programmed for specific tasks and cannot perform variations of them. <br>In the next chapter, you’ll learn what’s inside a **pipeline()** function and how to customize its behavior.